## 코드

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

In [2]:
# plt.rc('font', family='NanumGothicOTF') # For MacOS
plt.rc('font', family='NanumGothic') # For Windows
mpl.rcParams['axes.unicode_minus'] = False
pd.set_option('display.max_columns', None)

In [3]:
df1 = pd.read_csv('data_2017.csv',engine = 'python')

In [ ]:
df2 = pd.read_csv('data_2018.csv')

In [ ]:
print(df1.shape[0], df2.shape[0])

In [ ]:
frames = [df1, df2]
df = pd.concat(frames)

In [ ]:
df.shape[0]

In [ ]:
df = df.rename(columns={"연령대코드(5세단위)":"연령대 코드", 
                        "신장(5Cm단위)":"신장", 
                        "체중(5Kg단위)":"체중",
                        "식전혈당(공복혈당)":"식전혈당"})

In [ ]:
df.head()

In [ ]:
#필요 요소만 남김
df= df[['성별코드', '연령대 코드', '신장',
       '체중', '허리둘레', '수축기혈압',
       '이완기혈압', '식전혈당']].copy()

In [ ]:
df.describe()

## null 값 정리

In [ ]:
df.isnull().sum()

In [ ]:
#식전혈당이 없을 시 드랍
df = df.loc[df['식전혈당'].notnull()]

## 혈당에 따른 당뇨여부  

In [ ]:
df_diabetes = df['식전혈당'].copy()

In [ ]:
#당뇨병: 공복 혈당치 126 mg/dL 이상 당뇨병의 전단계: 공복 혈당치 100 mg/dL 이상
#저혈당: 혈당치 126 mg/dL 미만
df_diabetes.loc[df_diabetes < 100] = 0
df_diabetes.loc[(df_diabetes >= 100) & (df_diabetes < 126)] = 1
df_diabetes.loc[df_diabetes >= 126] = 2

In [ ]:
df['혈당'] = df_diabetes

In [ ]:
df = df.drop(['식전혈당'], axis = 1)

## 복부비만

In [ ]:
#비정상적인 값 배제
df = df.loc[df['허리둘레']<140]
df = df.loc[df['허리둘레']>40]

In [ ]:
#국내의 복부비만 기준은 남자는 90cm 이상, 여자의 경우 85cm 이상
df_obesity = df[['허리둘레','성별코드']].copy()

df_obesity.loc[(df_obesity['허리둘레']>= 90) & (df_obesity['성별코드']==1), '복부비만'] = 1
df_obesity.loc[(df_obesity['허리둘레']>= 85) & (df_obesity['성별코드']==2), '복부비만'] = 1
df_obesity['복부비만'] = df_obesity['복부비만'].fillna(0)

In [ ]:
df['복부비만'] = df_obesity['복부비만']

In [ ]:
df = df.drop(['허리둘레'], axis = 1)

## BMI

In [ ]:
df_weight = df[['신장', '체중']].copy()

In [ ]:
#체중(kg)을 신장(m)의 제곱으로 나눈 값(체중(kg)/신장(m2))
temp = df_weight['신장']/100
df_bmi = round(df_weight['체중'] / (temp * temp), 1)

In [ ]:
#세계보건기구 아시아태평양지역과 대한비만학회
#BMI < 25 kg/m2 : 저체중+정상 0
#BMI ≥ 25 kg/m2 : 과체중 1
#BMI ≥ 30 kg/m2 : 비만 2
df_bmi.loc[df_bmi < 25] = 0
df_bmi.loc[(df_bmi >= 25) & (df_bmi < 30)] = 1
df_bmi.loc[df_bmi >= 30] = 2

In [ ]:
df['비만여부'] = df_bmi

In [ ]:
df = df.drop(['체중', '신장'], axis = 1)

## 고혈압

In [ ]:
df_pressure = df[['이완기혈압', '수축기혈압']].copy()

In [ ]:
# 정상: 0 / 120~139/80~89 전단계: 1/  140~159/90~99 1기:2 / 160/100 2기:3
df_pressure.loc[(df_pressure['수축기혈압'] < 120) & (df_pressure['이완기혈압'] < 80), '고혈압 여부'] = 0
df_pressure.loc[((df_pressure['수축기혈압'] >= 120) & (df_pressure['수축기혈압'] < 140))|
                ((df_pressure['이완기혈압'] >= 80) & (df_pressure['이완기혈압'] < 90)), '고혈압 여부'] = 1
df_pressure.loc[((df_pressure['수축기혈압'] >= 140) & (df_pressure['수축기혈압'] < 160))|
                ((df_pressure['이완기혈압'] >= 90) & (df_pressure['이완기혈압'] < 99)), '고혈압 여부'] = 2
df_pressure.loc[(df_pressure['수축기혈압'] >=160) | (df_pressure['이완기혈압'] >= 100), '고혈압 여부'] = 3

In [ ]:
df_pressure.isnull().sum()

In [ ]:
df['고혈압'] = df_pressure['고혈압 여부']

In [ ]:
df = df.drop(['수축기혈압', '이완기혈압'], axis = 1)

## 최종

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df = df.loc[df['고혈압'].notnull()]

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
def get_plot(column_name):
    sns.countplot(x=column_name, hue='혈당', data=df, palette = "PuRd")
    plt.title(column_name)
    plt.show()
    
for c in df.columns:
    get_plot(c)